#Mining the Social Web, 2nd Edition

##Chapter 2: Mining Facebook: Analyzing Fan Pages, Examining Friendships, and More

This IPython Notebook provides an interactive way to follow along with and explore the numbered examples from [_Mining the Social Web (2nd Edition)_](http://bit.ly/135dHfs). The intent behind this notebook is to reinforce the concepts from the sample code in a fun, convenient, and effective way. This notebook assumes that you are reading along with the book and have the context of the discussion as you work through these exercises.

In the somewhat unlikely event that you've somehow stumbled across this notebook outside of its context on GitHub, [you can find the full source code repository here](http://bit.ly/16kGNyb).

## Copyright and Licensing

You are free to use or adapt this notebook for any purpose you'd like. However, please respect the [Simplified BSD License](https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/LICENSE.txt) that governs its use.

# Facebook API Access

Facebook implements OAuth 2.0 as its standard authentication mechanism, but provides a convenient way for you to get an _access token_ for development purposes, and we'll opt to take advantage of that convenience in this notebook. For details on implementing an OAuth flow with Facebook (all from within IPython Notebook), see the \_AppendixB notebook from the [IPython Notebook Dashboard](/).

For this first example, login to your Facebook account and go to https://developers.facebook.com/tools/explorer/ to obtain and set permissions for an access token that you will need to define in the code cell defining the ACCESS_TOKEN variable below.  

Be sure to explore the permissions that are available by clicking on the "Get Access Token" button that's on the page and exploring all of the tabs available. For example, you will need to set the "friends_likes" option under the "Friends Data Permissions" since this permission is used by the script below but is not a basic permission and is not enabled by default. 

<img src="files/resources/ch02-facebook/images/FB_GraphExplorer_perms.png" width="300px" /><br />

In [1]:
# Copy and paste in the value you just got from the inline frame into this variable and execute this cell.
# Keep in mind that you could have just gone to https://developers.facebook.com/tools/access_token/
# and retrieved the "User Token" value from the Access Token Tool

ACCESS_TOKEN = 'EAACEdEose0cBANGmeqoxZAvOMWwhKWSbIHtwSsRxMyi2cvhqYPJi6upMFSckGjgZAVHiamuZBfyNZAYZAOtE6uktRWr3N4ZAlIkZAdLOtGH5mIJZBmezneSvu79hG6OfiEKgGdhvMMy80Iuo5BZAfVvul7Tl73m2eEm4hGbFZAgZAgbrGB75hfplyywIQZAzLMOlobYZD'

## Example 1. Making Graph API requests over HTTP

In [2]:
import requests # pip install requests
import json

base_url = 'https://graph.facebook.com/me'

# Get 10 likes for 10 friends
fields = 'id,name,friends.limit(10).fields(likes.limit(10))'

url = '%s?fields=%s&access_token=%s' % \
    (base_url, fields, ACCESS_TOKEN,)

# This API is HTTP-based and could be requested in the browser,
# with a command line utlity like curl, or using just about
# any programming language by making a request to the URL.
# Click the hyperlink that appears in your notebook output
# when you execute this code cell to see for yourself...
print(url)

# Interpret the response as JSON and convert back
# to Python data structures
content = requests.get(url).json()

# Pretty-print(the JSON and display it
#print(json.dumps(content, indent=1)
print(json.dumps(content, indent=1))

https://graph.facebook.com/me?fields=id,name,friends.limit(10).fields(likes.limit(10))&access_token=EAACEdEose0cBANGmeqoxZAvOMWwhKWSbIHtwSsRxMyi2cvhqYPJi6upMFSckGjgZAVHiamuZBfyNZAYZAOtE6uktRWr3N4ZAlIkZAdLOtGH5mIJZBmezneSvu79hG6OfiEKgGdhvMMy80Iuo5BZAfVvul7Tl73m2eEm4hGbFZAgZAgbrGB75hfplyywIQZAzLMOlobYZD
{
 "id": "869985306428419",
 "name": "Igor Brand\u00e3o",
 "friends": {
  "data": [
   {
    "id": "577124531"
   },
   {
    "likes": {
     "data": [
      {
       "name": "Neta Salgados Finos",
       "category": "Kitchen/Cooking",
       "id": "1154355184676777",
       "created_time": "2017-04-03T00:33:53+0000"
      },
      {
       "name": "Os Melhores de Junho",
       "category": "Community",
       "id": "1823846184537221",
       "created_time": "2017-03-03T08:41:59+0000"
      },
      {
       "name": "NET",
       "category": "Telecommunication Company",
       "id": "122665237747800",
       "created_time": "2017-02-26T17:54:21+0000"
      },
      {
       "name": "Hail 

Note: If you attempt to run a query for all of your friends' likes and it appears to hang, it is probably because you have a lot of friends who have a lot of likes. If this happens, you may need to add limits and offsets to the fields in the query as described in Facebook's [field expansion](https://developers.facebook.com/docs/reference/api/field_expansion/) documentation. However, the <code>facebook</code> library that we'll use in the next example handles some of these issues, so it's recommended that you hold off and try it out first. This initial example is just to illustrate that Facebook's API is built on top of HTTP.

A couple of field limit/offset examples that illustrate the possibilities follow:

<code>
fields = 'id,name,friends.limit(10).fields(likes)'            # Get all likes for 10 friends 
fields = 'id,name,friends.offset(10).limit(10).fields(likes)' # Get all likes for 10 more friends 
fields = 'id,name,friends.fields(likes.limit(10))'            # Get 10 likes for all friends 
fields = 'id,name,friends.fields(likes.limit(10))'            # Get 10 likes for all friends
</code>

## Example 2. Querying the Graph API with Python

In [3]:
import facebook # pip install facebook-sdk
import json

# A helper function to pretty-print(Python objects as JSON

def pp(o): 
    print(json.dumps(o, indent=1))

# Create a connection to the Graph API with your access token

g = facebook.GraphAPI(ACCESS_TOKEN)

# Execute a few sample queries

print('---------------')
print('Me')
print('---------------')
pp(g.get_object('me'))
print()
print('---------------')
print('My Friends')
print('---------------')
pp(g.get_connections('me', 'friends'))
print()
print('---------------')
print('Social Web')
print('---------------')
pp(g.request("search", {'q' : 'social web', 'type' : 'page'}))

---------------
Me
---------------
{
 "id": "869985306428419",
 "birthday": "04/26/1992",
 "education": [
  {
   "school": {
    "id": "109098885803061",
    "name": "ITB - Instituto T\u00e9cnico de Barueri"
   },
   "type": "High School",
   "year": {
    "id": "136328419721520",
    "name": "2009"
   },
   "id": "105997129493911"
  },
  {
   "concentration": [
    {
     "id": "107953379227862",
     "name": "Information Technology"
    },
    {
     "id": "111213508903819",
     "name": "Embedded system"
    }
   ],
   "school": {
    "id": "110488412304653",
    "name": "Federal University of Rio Grande do Norte"
   },
   "type": "College",
   "year": {
    "id": "417640074981890",
    "name": "2018"
   },
   "id": "582057841887835"
  },
  {
   "school": {
    "id": "110262438997070",
    "name": "FATEC"
   },
   "type": "College",
   "year": {
    "id": "118118634930920",
    "name": "2012"
   },
   "id": "550522081708078"
  },
  {
   "concentration": [
    {
     "id": "152149264

## Example 3. Results for a Graph API query for Mining the Social Web

In [4]:
# Get an instance of Mining the Social Web
# Using the page name also works if you know it.
# e.g. 'MiningTheSocialWeb' or 'CrossFit'
mtsw_id = '146803958708175'
pp(g.get_object(mtsw_id))

{
 "id": "146803958708175",
 "about": "Analyzing Data from Facebook, Twitter, LinkedIn, and Other Social Media Sites. Twitter: @SocialWebMining",
 "awards": "Jolt Productivity Award (1st Ed.) -  http://drdobbs.com/joltawards/231500080?pgno=6",
 "can_post": true,
 "category": "Book",
 "checkins": 0,
 "company_overview": "Like It here on Facebook!\n\nFollow @SocialWebMining on Twitter: http://twitter.com/SocialWebMining\n\nGet the source code for the revised and expanded second edition on GitHub: http://bit.ly/MiningTheSocialWeb2E\n\nGet the (now legacy) source code for the first edition on GitHub: http://bit.ly/SocialWebMining",
 "cover": {
  "cover_id": "522831997772034",
  "offset_x": 0,
  "offset_y": 56,
  "source": "https://scontent.xx.fbcdn.net/v/t31.0-0/p180x540/1073953_522831997772034_741301400_o.jpg?oh=75ef006fae70db4df15d38169c8d8bbc&oe=59838B39",
  "id": "522831997772034"
 },
 "description": "Facebook, Twitter, LinkedIn, Google+, and other social web properties generate a weal

## Example 4. Querying the Graph API for Open Graph objects by their URLs

In [5]:
# MTSW catalog link
pp(g.get_object('http://shop.oreilly.com/product/0636920030195.do'))

# PCI catalog link
pp(g.get_object('http://shop.oreilly.com/product/9780596529321.do'))

{
 "og_object": {
  "id": "465351090213998",
  "description": "Facebook, Twitter, LinkedIn, Google+, and other social web properties generate a wealth of valuable social data, but how can you tap into this data and discover who\u2019s connecting with whom, which insights are lurking just beneath the surface,...",
  "title": "Mining the Social Web",
  "type": "book",
  "updated_time": "2017-04-20T10:58:16+0000"
 },
 "share": {
  "comment_count": 0,
  "share_count": 289
 },
 "id": "http://shop.oreilly.com/product/0636920030195.do"
}
{
 "og_object": {
  "id": "10150339462353143",
  "description": "This fascinating book demonstrates how you can build web applications to mine the enormous amount of data created by people on the Internet. With the sophisticated algorithms in this book, you can write smart programs to access interesting datasets...",
  "title": "Programming Collective Intelligence",
  "type": "book",
  "updated_time": "2017-04-16T22:25:57+0000"
 },
 "share": {
  "comment_coun

## Example 5. Comparing likes between Coke and Pepsi fan pages

In [6]:
# Find Pepsi and Coke in search results

pp(g.request('search', {'q' : 'pepsi', 'type' : 'page', 'limit' : 5}))
pp(g.request('search', {'q' : 'coke', 'type' : 'page', 'limit' : 5}))

# Use the ids to query for likes

pepsi_id = '56381779049' # Could also use 'PepsiUS'
coke_id = '40796308305'  # Could also use 'CocaCola'

# A quick way to format integers with commas every 3 digits
def int_format(n): return "{:,}".format(n)

print("Pepsi likes:", int_format(g.get_object(pepsi_id)['likes']))
print("Coke likes:", int_format(g.get_object(coke_id)['likes']))

{
 "data": [
  {
   "name": "Pepsi",
   "category": "Foodservice Distributor",
   "id": "112949768466"
  },
  {
   "name": "Pepsi Club",
   "category": "Dance & Night Club",
   "category_list": [
    {
     "id": "191478144212980",
     "name": "Dance & Night Club"
    }
   ],
   "id": "194989311723"
  },
  {
   "name": "PepsiCo Brasil",
   "category": "Product/Service",
   "id": "103802363017083"
  },
  {
   "name": "PepsiCo",
   "category": "Food & Beverage Company",
   "category_list": [
    {
     "id": "2252",
     "name": "Food & Beverage Company"
    }
   ],
   "id": "260431051694"
  },
  {
   "name": "Pepsi Center",
   "category": "Company",
   "category_list": [
    {
     "id": "109976259083543",
     "name": "Stadium, Arena & Sports Venue"
    }
   ],
   "id": "111829892187838"
  }
 ],
 "paging": {
  "cursors": {
   "before": "MAZDZD",
   "after": "NAZDZD"
  },
  "next": "https://graph.facebook.com/v2.3/search?access_token=EAACEdEose0cBANGmeqoxZAvOMWwhKWSbIHtwSsRxMyi2cvhqYPJ

## Example 6. Querying a page for its "feed" and "links" connections

In [19]:
pp(g.get_connections(pepsi_id, 'feed'))
pp(g.get_connections(pepsi_id, 'links'))

pp(g.get_connections(coke_id, 'feed'))
pp(g.get_connections(coke_id, 'links'))

{
 "data": [
  {
   "id": "56381779049_10155453602709050",
   "from": {
    "name": "Pepsi",
    "category": "Foodservice Distributor",
    "id": "56381779049"
   },
   "message": "This place is an institution, much like the cheese fry itself...well before we devour it anyway.",
   "picture": "https://scontent.xx.fbcdn.net/v/t1.0-0/p130x130/18056754_10155453602709050_3338174107236196870_n.jpg?oh=ca2fb8455da8be812eaf86dac9a2a951&oe=5975F842",
   "link": "https://www.facebook.com/PepsiUS/photos/a.123490744049.127432.56381779049/10155453602709050/?type=3",
   "name": "Timeline Photos",
   "icon": "https://www.facebook.com/images/icons/photo.gif",
   "actions": [
    {
     "name": "Comment",
     "link": "https://www.facebook.com/56381779049/posts/10155453602709050"
    },
    {
     "name": "Like",
     "link": "https://www.facebook.com/56381779049/posts/10155453602709050"
    },
    {
     "name": "Share",
     "link": "https://www.facebook.com/56381779049/posts/10155453602709050"
    }

## Example 7. Querying for all of your friends' likes

In [7]:
# First, let's query for all of the likes in your social
# network and store them in a slightly more convenient
# data structure as a dictionary keyed on each friend's
# name. We'll use a dictionary comprehension to iterate
# over the friends and build up the likes in an intuitive
# way, although the new "field expansion" feature could 
# technically do the job in one fell swoop as follows:
#
# g.get_object('me', fields='id,name,friends.fields(id,name,likes)')
#
# See Appendix C for more information on Python tips such as
# dictionary comprehensions

friends = g.get_connections("me", "friends")['data']

likes = { friend['name'] : g.get_connections(friend['id'], "likes")['data'] 
          for friend in friends }

print(likes)

{'Ibrahim Cesar': [], 'Valquíria Andrade': [{'name': 'Neta Salgados Finos', 'category': 'Kitchen/Cooking', 'id': '1154355184676777', 'created_time': '2017-04-03T00:33:53+0000'}, {'name': 'Os Melhores de Junho', 'category': 'Community', 'id': '1823846184537221', 'created_time': '2017-03-03T08:41:59+0000'}, {'name': 'NET', 'category': 'Telecommunication Company', 'id': '122665237747800', 'created_time': '2017-02-26T17:54:21+0000'}, {'name': 'Hail Construtora', 'category': 'Company', 'category_list': [{'id': '108427109235243', 'name': 'Home Improvement'}, {'id': '176831012360626', 'name': 'Professional Service'}], 'id': '549595338385094', 'created_time': '2017-02-07T23:35:17+0000'}, {'name': 'Gramado - Serra Gaúcha/RS', 'category': 'Eco Tour Agency', 'category_list': [{'id': '214416301907154', 'name': 'Eco Tour Agency'}], 'id': '159387964137804', 'created_time': '2017-02-07T22:38:10+0000'}, {'name': 'Cultura Inglesa - RN', 'category': 'Education', 'id': '822740741103594', 'created_time': 

## Example 8. Calculating the most popular likes among your friends

In [8]:
# Analyze all likes from friendships for frequency

# pip install prettytable
from prettytable import PrettyTable
from collections import Counter
friends_likes = Counter([like['name']
                         for friend in likes 
                           for like in likes[friend]
                               if like.get('name')])

pt = PrettyTable(field_names=['Name', 'Freq'])
pt.align['Name'], pt.align['Freq'] = 'l', 'r'
[ pt.add_row(fl) for fl in friends_likes.most_common(10) ]

print('Top 10 likes amongst friends')
print(pt)

ModuleNotFoundError: No module named 'prettytable'

## Example 9. Calculating the most popular categories for likes among your friends

In [10]:
# Analyze all like categories by frequency

friends_likes_categories = Counter([like['category'] 
                                    for friend in likes 
                                      for like in likes[friend]])

pt = PrettyTable(field_names=['Category', 'Freq'])
pt.align['Category'], pt.align['Freq'] = 'l', 'r'
[ pt.add_row(flc) for flc in friends_likes_categories.most_common(10) ]

print("Top 10 like categories for friends")
print(pt)

NameError: name 'Counter' is not defined

## Example 10. Calculating the number of likes for each friend and sorting by frequency

In [12]:
# Build a frequency distribution of number of likes by 
# friend with a dictionary comprehension and sort it in 
# descending order

from operator import itemgetter

num_likes_by_friend = { friend : len(likes[friend]) 
                        for friend in likes }


pt = PrettyTable(field_names=['Friend', 'Num Likes'])
pt.align['Friend'], pt.align['Num Likes'] = 'l', 'r'
[ pt.add_row(nlbf) 
  for nlbf in sorted(num_likes_by_friend.items(), 
                     key=itemgetter(1), 
                     reverse=True) ]

print("Number of likes per friend")
print(pt)

NameError: name 'PrettyTable' is not defined

## Example 11. Finding common likes between an ego and its friendships in a social network

In [15]:
# Which of your likes are in common with which friends?
my_likes = [ like['name'] 
             for like in g.get_connections("me", "likes")['data'] ]

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[ pt.add_row((ml,)) for ml in my_likes ]
print("My likes")
print(pt)

# Use the set intersection as represented by the ampersand
# operator to find common likes.

common_likes = list(set(my_likes) & set(friends_likes))

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[ pt.add_row((cl,)) for cl in common_likes ]
print()
print("My common likes with friends")
print(pt)

NameError: name 'PrettyTable' is not defined

## Example 12. Calculating the friends most similar to an ego in a social network

In [16]:
# Which of your friends like things that you like?

similar_friends = [ (friend, friend_like['name']) 
                     for friend, friend_likes in likes.items()
                       for friend_like in friend_likes
                         if friend_like.get('name') in common_likes ]


# Filter out any possible duplicates that could occur

ranked_friends = Counter([ friend for (friend, like) in list(set(similar_friends)) ])


pt = PrettyTable(field_names=["Friend", "Common Likes"])
pt.align["Friend"], pt.align["Common Likes"] = 'l', 'r'
[ pt.add_row(rf) 
  for rf in sorted(ranked_friends.items(), 
                   key=itemgetter(1), 
                   reverse=True) ]
print("My similar friends (ranked)")
print(pt)

# Also keep in mind that you have the full range of plotting
# capabilities available to you. A quick histogram that shows
# how many friends.

plt.hist(ranked_friends.values())
plt.xlabel('Bins (number of friends with shared likes)')
plt.ylabel('Number of shared likes in each bin')

# Keep in mind that you can customize the binning
# as desired. See http://matplotlib.org/api/pyplot_api.html

# For example...

plt.figure() # Display the previous plot
plt.hist(ranked_friends.values(),
          bins=arange(1,max(ranked_friends.values()),1))
plt.xlabel('Bins (number of friends with shared likes)')
plt.ylabel('Number of shared likes in each bin')
plt.figure() # Display the working plot

NameError: name 'common_likes' is not defined

## Example 13. Constructing a graph of mutual friendships

In [18]:
import networkx as nx # pip install networkx
import requests # pip install requests

friends = [ (friend['id'], friend['name'],)
                for friend in g.get_connections('me', 'friends')['data'] ]

url = 'https://graph.facebook.com/me/mutualfriends/%s?access_token=%s'

mutual_friends = {} 

# This loop spawns a separate request for each iteration, so
# it may take a while. Optimization with a thread pool or similar
# technique would be possible.
for friend_id, friend_name in friends:
    r = requests.get(url % (friend_id, ACCESS_TOKEN,) )
    response_data = json.loads(r.content)['data']
    mutual_friends[friend_name] = [ data['name'] 
                                    for data in response_data ]
    
nxg = nx.Graph()

[ nxg.add_edge('me', mf) for mf in mutual_friends ]

[ nxg.add_edge(f1, f2) 
  for f1 in mutual_friends 
      for f2 in mutual_friends[f1] ]

# Explore what's possible to do with the graph by 
# typing nxg.<tab> or executing a new cell with 
# the following value in it to see some pydoc on nxg
print(nxg)

KeyError: 'data'

## Example 14. Finding and analyzing cliques in a graph of mutual friendships

In [19]:
# Finding cliques is a hard problem, so this could
# take a while for large graphs.
# See http://en.wikipedia.org/wiki/NP-complete and 
# http://en.wikipedia.org/wiki/Clique_problem.

cliques = [c for c in nx.find_cliques(nxg)]

num_cliques = len(cliques)

clique_sizes = [len(c) for c in cliques]
max_clique_size = max(clique_sizes)
avg_clique_size = sum(clique_sizes) / num_cliques

max_cliques = [c for c in cliques if len(c) == max_clique_size]

num_max_cliques = len(max_cliques)

max_clique_sets = [set(c) for c in max_cliques]
friends_in_all_max_cliques = list(reduce(lambda x, y: x.intersection(y),
                                  max_clique_sets))

print('Num cliques:', num_cliques)
print('Avg clique size:', avg_clique_size)
print('Max clique size:', max_clique_size)
print('Num max cliques:', num_max_cliques)
print()
print('Friends in all max cliques:')
print(json.dumps(friends_in_all_max_cliques, indent=1))
print()
print('Max cliques:')
print(json.dumps(max_cliques, indent=1))

NameError: name 'nxg' is not defined

## Example 15. Serializing a NetworkX graph to a file for consumption by D3

In [20]:
from networkx.readwrite import json_graph

nld = json_graph.node_link_data(nxg)

json.dump(nld, open('resources/ch02-facebook/viz/force.json','w'))

NameError: name 'nxg' is not defined

Note: You may need to implement some filtering on the NetworkX graph before writing it out to a file for display in D3, and for more than dozens of nodes, it may not be reasonable to render a meaningful visualization without some JavaScript hacking on its parameters. View the JavaScript source in [force.html](files/resources/ch02-facebook/viz/force.html) for some of the details.

## Example 16. Visualizing a mutual friendship graph with D3

In [22]:
from IPython.display import IFrame
from IPython.core.display import display

# IPython Notebook can serve files and display them into
# inline frames. Prepend the path with the 'files' prefix.

viz_file = 'files/resources/ch02-facebook/viz/force.html'

display(IFrame(viz_file, '100%', '600px'))